In [10]:
import mediapipe as mp
import os
import numpy as np
import cv2
import uuid

In [13]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [21]:
def calc_bounding_rect(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv2.boundingRect(landmark_array)

    return [x, y, x + w, y + h]

In [26]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make Detections
        results = holistic.process(image)
        image.flags.writeable = True
        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        # 2. Right hand
        if results.right_hand_landmarks: 
            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                     mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2),
                                     mp_drawing.DrawingSpec(color=(255,255,174), thickness=2, circle_radius=1)
                                     )
            b_rect_right = calc_bounding_rect(image,results.right_hand_landmarks)
            cv2.rectangle(image,(b_rect_right[0],b_rect_right[1]),(b_rect_right[2],b_rect_right[3]),(255,255,255),2)
            cv2.rectangle(image, (b_rect_right[0], b_rect_right[1]), (b_rect_right[2], b_rect_right[1] - 22),
                 (0, 0, 0), -1)
            cv2.putText(image, "right", (b_rect_right[0] + 5, b_rect_right[1] - 4),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0, 255), 1, cv2.LINE_AA)
        # 3. Left Hand
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                     mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2),
                                     mp_drawing.DrawingSpec(color=(255,255,174), thickness=2, circle_radius=1)
                                     )
            b_rect_left = calc_bounding_rect(image,results.left_hand_landmarks)
            cv2.rectangle(image,(b_rect_left[0],b_rect_left[1]),(b_rect_left[2],b_rect_left[3]),(255,255,255),2)
            cv2.rectangle(image, (b_rect_left[0], b_rect_left[1]), (b_rect_left[2], b_rect_left[1] - 22),
                 (0, 0, 0), -1)
            cv2.putText(image, "left", (b_rect_left[0] + 5, b_rect_left[1] - 4),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 1, cv2.LINE_AA)
        cv2.imshow('Raw Webcam Feed', image)


        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [20]:
cap.release()
cv2.destroyAllWindows()  